In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
%matplotlib inline

In [8]:
# Set path
path_root = '/scratch/yassinebha/pleio/pleio_all_tasks/'
path_pleio = os.path.join(path_root,'pleio_2018-07-26')


In [9]:
# Load variables
## Spatial subtypes from all tasks (90 tasks)
spatial_subt_df = pd.read_csv(os.path.join(path_root,'all_subtype_stack.csv'))

## Subjects weigts from all subtypes tasks  (90 tasks)
weight_subt_df = pd.read_csv(os.path.join(path_root,'subtype_weights_all_tasks.csv'))

## Subject weights and covariates (age,gender,FD,BMI)
weight_subt_cov_df = pd.read_csv(os.path.join(path_root,'pleio_all_pheno.csv'))

## Genetic correlation matrix of subtypes weights
Rho_G = np.load(os.path.join(path_pleio, 'RhoG.npy'))

## FDR of genetic corrrelation matrix
Rho_G_fdr =  np.load(os.path.join(path_pleio, 'RhoG_pval_0.npy'))
